##### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import imageio
from PIL import Image
import ipyplot
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score, precision_score, recall_score, f1_score
import pathlib

### DO NOT RUN BELOW YOU NEED TO HAVE THE 158 GB FOLDER
* I am using a subset of data from the 158 GB dataset

##### Exploring the data

In [ ]:
data_dir = "../input/hpa-single-cell-image-classification/train"
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = os.listdir(data_dir)
len(image_count)

In [ ]:
df_train = pd.read_csv("../input/hpa-single-cell-image-classification/train.csv")
df_train.head()

In [ ]:
train_images = "../input/hpa-single-cell-image-classification/train"
train_images = os.listdir(train_images)

### Note
*  the following code below shows that there are 4 images in the train folder assoicated with the image ID that corresponds to Red, Blue, Green, and Yellow channels...In other words, 4 images per ID that are differentiated by color (Red, Blue, Green, Yellow)

In [ ]:
len(df_train), len(train_images), len(train_images)/len(df_train)

##### Number of labels per images
* We can see that there are more than 10,000 images that are composed of 1 label out of the 18 labels so due to space constraints we will stick with the 1 label images

In [ ]:
df_train['num_classes'] = df_train['Label'].apply(lambda r: len(r.split('|')))
df_train['num_classes'].value_counts().plot.bar(title='Examples with multiple labels', xlabel='number of labels per example', ylabel='# train examples')
plt.show()

##### Things to keep in mind regarding channels
* Red (Microtubules)
* Green (Protein of interest)
* Blue (Nucleus)
* Yellow (Endoplasmic reticulum)

##### Here we seperate the images based on the ID and color

In [ ]:
path = "../input/hpa-single-cell-image-classification"
mt = [path+'/train/'+(id+'_red.png') for id in df_train.ID.values]
er = [path+'/train/'+(id+'_yellow.png') for id in df_train.ID.values]
nu = [path+'/train/'+(id+'_blue.png') for id in df_train.ID.values]
pr = [path+'/train/'+(id+'_green.png') for id in df_train.ID.values]
images = [mt, er, nu, pr]
titles = ['microtubules', 'endoplasmic reticulum', 'nucleus', 'protein of interest']

##### Viewing the images 
* Images by themselves (no stack)

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,12))
for j in range(3):
    for i in range(4):
        img = plt.imread(images[i][j])
        if j == 0: ax[j,i].set_title(titles[i])
        ax[j,i].imshow(img)
        ax[j,i].axis('off')
plt.show()

##### Generating stacked Images
* These are images stacked (red, yellow, blue)

In [ ]:
labels = [str(i) for i in range(19)]
class_images = []
for label in labels:
    r_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[0] + '_red.png'
    y_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[0] + '_yellow.png'
    b_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[0] + '_blue.png'
    r = imageio.imread(path+'/train/'+r_img)
    y = imageio.imread(path+'/train/'+y_img)
    b = imageio.imread(path+'/train/'+b_img)
    rgb = np.dstack((r,y,b))
    class_images.append(PIL.Image.frombytes('RGB',r.shape,rgb))

codes = [
'0. Nucleoplasm',
'1. Nuclear membrane',
'2. Nucleoli',
'3. Nucleoli fibrillar center',
'4. Nuclear speckles',
'5. Nuclear bodies',
'6. Endoplasmic reticulum',
'7. Golgi apparatus',
'8. Intermediate filaments',
'9. Actin filaments',
'10. Microtubules',
'11. Mitotic spindle',
'12. Centrosome',
'13. Plasma membrane',
'14. Mitochondria',
'15. Aggresome',
'16. Cytosol',
'17. Vesicles and punctate cytosolic patterns',
'18. Negative'
]

ipyplot.plot_images(images=class_images, labels=codes, max_images=19, img_width=300)

##### Generating directories and saving images with respect to the labels associated to the images

In [ ]:
# 25 images 
img_counter = list(range(25))
# per label 
labels = [str(i) for i in range(19)]
code = [
'00_Nucleoplasm',
'01_Nuclear_membrane',
'02_Nucleoli',
'03_Nucleoli_fibrillar_center',
'04_Nuclear_speckles',
'05_Nuclear_bodies',
'06_Endoplasmic_reticulum',
'07_Golgi_apparatus',
'08_Intermediate_filaments',
'09_Actin_filaments',
'10_Microtubules',
'11_Mitotic_spindle',
'12_Centrosome',
'13_Plasma_membrane',
'14_Mitochondria',
'15_Aggresome',
'16_Cytosol',
'17_Vesicles_and_punctate_cytosolic_patterns',
'18_Negative'
]

for label in labels:
    print("Generating Images for "+code[int(label)] )
    for i in img_counter:
        try:
            r_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[i] + '_red.png'
            y_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[i] + '_yellow.png'
            b_img = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[i] + '_blue.png'
            name = df_train[df_train.Label == label].reset_index(drop=True).ID.loc[i]

            r = imageio.imread(path+'/train/'+r_img)
            y = imageio.imread(path+'/train/'+y_img)
            b = imageio.imread(path+'/train/'+b_img)
            rgb = np.dstack((r,y,b))
            image_name = PIL.Image.frombytes('RGB',r.shape,rgb)


            if not os.path.exists("data/"+code[int(label)]):
                os.makedirs("data/"+code[int(label)])

            image_name.save("data/"+code[int(label)]+"/"+ name + ".png")
        except:
            print("Unable to save image with this label: "+ code[int(label)])
    
    print("Completed Generating Images for "+code[int(label)] )


### RUN ALL BELOW

##### Size of images and batch size variables

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

##### Training Data Generator

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
 "./data",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

##### Validation Data Generator

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./data",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

##### Training Data

In [ ]:
x_train = list()
y_train = list()
for raw_data,labels in train_ds:
    y_train.extend(np.array(labels))
   # print(labels)
    for image_data_per in raw_data:
        x_train.append(np.array(image_data_per))
x_train = np.array(x_train).reshape(-1,(180*180*3))
#print(x_train.shape)
y_train = np.array(y_train)
#print(y_train.shape)

##### Validation data

In [ ]:
x_test = list()
y_test = list()
for raw_data,labels in val_ds:
    y_test.extend(np.array(labels))
    #print(labels)
    for image_data_per in raw_data:
        x_test.append(np.array(image_data_per))
x_test = np.array(x_test).reshape(-1,(180*180*3))
#print(x_test.shape)
y_test = np.array(y_test)
#print(y_test.shape)

**Create functions to create multi-label AUC_ROC curve**

In [ ]:
import sklearn.metrics as metrics
import seaborn as sns
def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.decision_function(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    label_ = dict(code)
    
    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (AUC = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

In [ ]:
def plot_multiclass_roc_advanced(clf, X_test, y_test, n_classes, figsize=(17, 6)):
    y_score = clf.predict_proba(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (AUC = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

##### SVM Classifer

In [ ]:
pipe = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test,y_pred))
precision = precision_score(y_test, y_pred,average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred,average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(y_test, y_pred,average='weighted')
print('F1 score: %f' % f1)

In [ ]:
plot_multiclass_roc(pipe, x_test, y_test, n_classes=18, figsize=(16, 10))

##### Random Forest Classifer

In [ ]:
pipe = make_pipeline(StandardScaler(), RandomForestClassifier())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test,y_pred))
precision = precision_score(y_test, y_pred,average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred,average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(y_test, y_pred,average='weighted')
print('F1 score: %f' % f1)


In [ ]:
plot_multiclass_roc_advanced(pipe, x_test, y_test, n_classes=18, figsize=(16, 10))

#### XGBoost Classifer

In [ ]:
pipe = make_pipeline(StandardScaler(), XGBClassifier())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test,y_pred))
precision = precision_score(y_test, y_pred,average='weighted')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred,average='weighted')
print('Recall: %f' % recall)
# f1: tp / (tp + fp + fn)
f1 = f1_score(y_test, y_pred,average='weighted')
print('F1 score: %f' % f1)

In [ ]:
plot_multiclass_roc_advanced(pipe, x_test, y_test, n_classes=18, figsize=(16, 10))